# Example of mean seasonal cycle and filter 


In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import glob
import matplotlib.pyplot as plt
%matplotlib inline  
#from matplotlib import animation
import cartopy.crs as ccrs
from scipy.io import netcdf
from scipy import signal
from netCDF4 import num2date
from netCDF4 import Dataset as NetCDFFile 
import matplotlib.dates as mdates
from mpl_toolkits.basemap import Basemap
import datetime
from wavelets import WaveletAnalysis

In [ ]:

path='/home/garcia/Documents/PhD/DATA/'
filenames=sorted(glob.glob('/home/garcia/Documents/PhD/DATA/ENTROPY/day5_average/*.nc'))
mfiles=sorted(glob.glob('/home/garcia/Documents/PhD/DATA/GOM/*.nc'))
print(len(filenames))
meanfile='/home/garcia/Documents/PhD/DATA/ENTROPY/Smean.nc'


ds = xr.open_mfdataset(filenames,concat_dim='time') #files of  time series of entropy 
dsl=xr.open_dataset(path+'GOM/GOM025-GSL301.041_y1993-2012.1d_gridT.nc')# file for the lats, lons and time
mds=xr.open_dataset(meanfile) # file of mean  Entropy 
lats=dsl.nav_lat 
lons=dsl.nav_lon
time2=dsl.time_centered #time of SSH (daily)
time3 = pd.date_range('1996-12-31','2012-12-26' , freq='5D') # time of the entropy (evry 5 days)

In [6]:
def clim_stS(smean,yi):
    ## Mean seasonal cycle for series time with delta T of 5 days 
    ## smean: series time
    ## yi year of start of the series time 
    ## smep: in the out of the mean seasonal cycle repeated every year, so is the same shape as smean
    ## sme1: is the out of just one year of the seasonal cycle
    if yi==1997:
        l=0
        fy=14
        ny=15
        diy=[73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73]
        years=[1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012]
    else:
        l=0
        fy=18
        ny=19
        diy=[73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73]
        years=[1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012]
        
    
    csdiy=np.cumsum(diy)
    
    smeassh=np.empty(smean.shape)
    smeassh[:]=np.NaN
    smep=np.empty(len(smean))
    smep[:]=np.NaN
    
    for j in range(73):
        
        smeassh[l]=smean[l]
        smep[l]=smeassh[l]
        
        for k in range(ny):
                        
            smeassh[l]=smeassh[l]+smean[l+csdiy[k]]
               
                        
    
            dd=dd+1
        #print(j+csdiy[:-1])
        #print(l,csdiy[:-1])
        smep[l+csdiy[:-1]]=smeassh[l]
        smep[l]=smeassh[l]
            
        l=l+1
    sme1=smeassh[:73]
    return (smep,sme1)

In [7]:
##use of the fuction
mlo=43
mla=87
st=np.squeeze(ds.S[292:,mla,mlo])#series time at one point,index 292 in my series time represents the begining of 1997
pclim,mclim=clim_stS(st.values,1997)# 

NameError: name 'ds' is not defined

In [ ]:
##For the filter
windows=[19,37]
win=windows[0]
xpad=np.pad(st.values,win/2,mode='mean')
#I think at hte end i dont use the pad for this method
deltat=5 
fs=1./win #frequency of filter
b,a=signal.butter(3,fs)
rx=signal.filtfilt(b,a,st.values)

In [ ]:
###I add the fucntion to calculate the mean seasonal cycle for dayly data, its similar of the other one, 
##except it have a few  tricks for the february 29, in this case i just ignore the 29 february, it means every 4 years 
## I skip this day but basically calculates the same: mean seasonal cycle repeated n- times, 
##to have the same length as my original series, and the seasonal cycle for just one year.

In [ ]:
def clim_st2(smean,yi):
    
    if yi==1997:
        l=1
        fy=14
        ny=15
        diy=[365,365,365,366,365,365,365,366,365,365,365,366,365,365,365,366]
        years=[1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012]
    else:
        l=0
        fy=18
        ny=19
        diy=[365,365,365,366,365,365,365,366,365,365,365,366,365,365,365,366,365,365,365,366]
        years=[1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012]
        
    print(len(diy),len(years))
    csdiy=np.cumsum(diy)
    dd=365
    smeassh=np.empty(len(smean))
    smeassh[:]=np.NaN
    smep=np.empty(len(smean))
    smep[:]=np.NaN
    #print(csdiy, smeassh.shape)
    for j in range(365):
        smeassh[l]=smean[l]
        smep[l]=smeassh[l]
        #print(smean[l],smeassh[l])
        for k in range(ny):
            #k=kl+1
            #print(l+csdiy[k])
            if l>59 and l<360:
                if k==2 or k==6 or k==10 or k==14 or k==18:

                    smeassh[l]=smeassh[l]+smean[l+csdiy[k]+1]
     
                else:

                    smeassh[l]=smeassh[l]+smean[l+csdiy[k]]
               
            elif l>359 and k==fy:
                 smeassh[l]=smeassh[l]+smean[l+csdiy[k-1]+1]
                #print('final year is %s' %k)
            
            else:
            
            
                smeassh[l]=smeassh[l]+smean[l+csdiy[k]]
             
    
            dd=dd+1
        #print(j+csdiy[:-1])
        if l<360:
            smep[l+csdiy[:-1]]=smeassh[l]
            smep[l]=smeassh[l]
            #print(smeassh[l],smep[l+csdiy[:-1]],l,j)
        else:
            smep[l+csdiy[:-2]]=smeassh[l]
            smep[l]=smeassh[l]
            #print(smeassh[l],smep[l+csdiy[:-2]],l,j)
 
        l=l+1
    sme1=smeassh[:365]
    return (smep,sme1)